In [ ]:
from google.colab import drive
drive.mount('/content/drive')


MessageError: Error: credential propagation was unsuccessful

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = load_model('/content/drive/MyDrive/models/model2.keras')

In [ ]:
model.get_config()

{'name': 'functional_1',
 'trainable': True,
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_shape': (None, 72, 33, 33, 4),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_layer_1'},
   'registered_name': None,
   'name': 'input_layer_1',
   'inbound_nodes': []},
  {'module': 'keras.layers',
   'class_name': 'TimeDistributed',
   'config': {'name': 'time_distributed',
    'trainable': True,
    'dtype': {'module': 'keras',
     'class_name': 'DTypePolicy',
     'config': {'name': 'float32'},
     'registered_name': None},
    'layer': {'module': 'keras',
     'class_name': 'Sequential',
     'config': {'name': 'sequential',
      'trainable': True,
      'dtype': {'module': 'keras',
       'class_name': 'DTypePolicy',
       'config': {'name': 'float32'},
       'registered_name': None},
      'layers': [{'module': 'keras.layers',
        'class_name': 'InputLayer',
        'config': {'batch_shape': (None, 33, 33, 4),
         

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 72, 33, 33, 4)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 72, 128)             │       1,142,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ yield_output (Dense)                 │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,451,685 (13.17 MB)

 Trainable params: 1,150,561 (4.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,301,124 (8.78 MB)

In [ ]:
# Step 2: Freeze certain layers
for layer in model.layers[:-2]:  # Freeze all layers except the last two
    layer.trainable = True

In [ ]:
# Step 3: Modify the output layer
# Assuming your task is regression (adjust for classification tasks)
new_output = Dense(1, activation='linear', name='yield_output')(model.layers[-2].output)

In [ ]:
# Create a new model
fine_tuned_model = tf.keras.Model(inputs=model.input, outputs=new_output)

# Step 4: Compile the model
fine_tuned_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Use a lower learning rate for fine-tuning
    loss=MeanSquaredError(),            # Use RMSE for regression tasks
    metrics=['mae']                     # Add additional metrics as needed
)

# Step 5: Fine-tune the model with your dataset
# Replace 'train_data' and 'val_data' with your data generators or datasets
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = fine_tuned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,                           # Fine-tune for a reasonable number of epochs
    callbacks=[early_stopping],
    batch_size=32                        # Adjust batch size as needed
)

# Step 6: Save the fine-tuned model
fine_tuned_model.save('fine_tuned_model.keras')

# Optional: Evaluate the model
results = fine_tuned_model.evaluate(test_data)
print(f"Test Loss: {results[0]}, Test MAE: {results[1]}")

NameError: name 'train_data' is not defined

In [ ]:
# Satellite data

import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array




# Define the paths
base_path = "/content/drive/MyDrive/GEE_Images/Seasonal_Images"
years = ["2019", "2020", "2021", "2022", "2023"]
seasons = ["Yala", "Maha"]

cnn_input_shape = (256, 256, 4)  # Adjust this based on your CNN model's input shape

# Function to load and preprocess images
def load_images_from_folder(folder_path, input_shape, expected_count):
    images = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith('.npy'):
            img_path = os.path.join(folder_path, filename)
            img = np.load(img_path)
            img = np.nan_to_num(img, nan=0.0)
            if img.shape != input_shape:
                print(f"Warning: Image {filename} has shape {img.shape}, expected {input_shape}")
                img = np.resize(img, input_shape)
            images.append(img)
    # Handle cases with missing images by padding or discarding
    if len(images) < expected_count:
        # Pad with zeros if fewer images than expected
        while len(images) < expected_count:
            padding_image = np.zeros(input_shape)
            images.append(padding_image)
    elif len(images) > expected_count:
        # Trim the excess images if more than expected
        images = images[:expected_count]

    return np.array(images)

# Function to prepare data for the model
def prepare_data_for_model(base_path, years, seasons, input_shape):
    all_season_data = []
    for year in years:
        for season in seasons:
            folder_path = os.path.join(base_path, year, season)
            expected_count = 6 if season == "Maha" else 6  # Adjust expected count based on season
            images = load_images_from_folder(folder_path, input_shape, expected_count)
            all_season_data.append(images)
    return all_season_data  # Return as a list of arrays instead of a single array

satellite_placeholder = np.zeros((6, 1024, 1024, 4))

# Prepare the satellite data
satellite_input = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Hambanthota/Seasonal_Images_Hambanthota", years, seasons, cnn_input_shape)
# satellite_input.append(satellite_placeholder)

satellite_input_2 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Polonnaruwa/Seasonal_Images_Polonnaruwa", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_2)
# satellite_input.append(satellite_placeholder)

satellite_input_3 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Mannar/Seasonal_Images_Mannar", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_3)
# satellite_input.append(satellite_placeholder)

satellite_input_4 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Trincomalee/Seasonal_Images_Trincomalee", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_4)
# satellite_input.append(satellite_placeholder)

satellite_input_5 = prepare_data_for_model("/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/GEE_Images/GEE_Images_Badulla/Seasonal_Images_Badulla", years, seasons, cnn_input_shape)
satellite_input.extend(satellite_input_5)

In [ ]:
def mask_zeros(input_data):
    # Create a mask where the value is 0 if all channels are zero at a pixel
    mask = np.any(input_data != 0, axis=-1)

    # Expand dimensions to match the input data shape
    mask = np.expand_dims(mask, axis=-1)

    # Apply the mask to the input data
    masked_input_data = input_data * mask

    return masked_input_data

satellite_input = np.array([mask_zeros(sequence) for sequence in satellite_input])

In [ ]:
# Yeild data

import pandas as pd
import numpy as np

# Load yield data
yield_data = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_hambantota_yield.csv')
yield_data1 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_polonnaruwa_yield.csv')
yield_data2 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_mannar_yield.csv')
yield_data3 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_trincomalee_yield.csv')
yield_data4 = pd.read_csv('/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/filtered_badulla_yield.csv')


def create_y_target(yield_data):
    # Ensure the data is sorted by Year and District (if needed)
    yield_data = yield_data.sort_values(by=['Year', 'District'])

    # Extract yield values and map to seasons
    years = ["2019", "2020", "2021", "2022", "2023"]
    seasons = ["Yala", "Maha"]

    Y_target = []

    for year in years:
        # Filter data for the current year
        year_data = yield_data[yield_data['Year'] == int(year)]

        # Extract Yala and Maha yields
        yala_yield = year_data.iloc[0]['Yield']
        maha_yield = year_data.iloc[1]['Yield']

        # Append to Y_target
        Y_target.append(yala_yield)
        Y_target.append(maha_yield)

    # Convert Y_target to numpy array
    # Y_target.append(0)
    Y_target = np.array(Y_target)
    return Y_target

Y_target1 = create_y_target(yield_data)
Y_target2 = create_y_target(yield_data1)
Y_target3 = create_y_target(yield_data2)
Y_target4 = create_y_target(yield_data3)
Y_target5 = create_y_target(yield_data4)

Y_target = np.concatenate((Y_target1, Y_target2))
Y_target = np.concatenate((Y_target, Y_target3))
Y_target = np.concatenate((Y_target, Y_target4))
Y_target = np.concatenate((Y_target, Y_target5))
# Y_target = Y_target[:-1]
Y_target = Y_target*1000*0.404686
print(Y_target)

[2679.82203617 2317.28017421 2550.45923003 2422.29913044 2380.82762505
 2352.72251743 1606.31745511 1866.58856121 2212.76128725 2058.60540505
 2053.71162345 2230.22648242 2186.97648    2122.35815189 2112.55637329
 2201.74433331 1255.6830813  1735.42527711 1787.22812632 1990.36186779
 2068.85708158 2232.03103112 1953.46562941 2174.56192191 2115.81724283
 1995.49921775 1077.4536236  1385.92372624 1415.31802951 1953.29873933
 1716.20971907 2014.52469356 1770.34622289 2132.01870763 1719.73780966
 2131.40610003 1077.31227568 1629.36273847 1270.06261088 1930.70878155
 1905.33075576 2071.82767269 1782.82797261 2012.58983111 1966.44306809
 2059.12138496 1224.88532415 1355.41832843 1481.85811471 1842.87500529]


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the ground data for different locations
ground_data_paths = [
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Polonnaruwa.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Mannar.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Trincomalee.csv",
    "/content/drive/MyDrive/FYP20-Satellite Image Analysis/Data sets/Weather Data/monthly_weather_data_Badulla.csv"
]

ground_dfs = [pd.read_csv(path) for path in ground_data_paths]

# Extract relevant columns (adjust as per your needs)
features = ['temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'precipitation_sum (mm)', 'wind_speed_10m_max (km/h)']  # Example features

# Normalize the data
scaler = MinMaxScaler()
# for df in ground_dfs:
#     df[features] = scaler.fit_transform(df[features])

# Define the months for each season (use months 1 and 2 from the next year for Maha)
season_months = {
    "Yala": [3, 4, 5, 6, 7, 8],
    "Maha": [9, 10, 11, 12, 1, 2]
}

# Prepare ground data, moving months 1 and 2 to the previous year's Maha season
def prepare_ground_data(df, years, seasons):
    ground_data = []
    for year in years:
        for season in seasons:
            if season == 'Maha':
                # Maha includes months from both the current and next year
                maha_df = df[((df['year'] == int(year)) & (df['month'].isin([9, 10, 11, 12]))) |
                             ((df['year'] == int(year) + 1) & (df['month'].isin([1, 2])))]
            else:
                # Yala includes months within the same year
                maha_df = df[(df['year'] == int(year)) & (df['month'].isin([3, 4, 5, 6, 7, 8]))]

            # Extract relevant features for the season
            ground_data.append(maha_df[features].values)
    return ground_data

# Prepare the ground data for all years and seasons
years = [2019, 2020, 2021, 2022, 2023]  # List of years to process
seasons = ["Yala", "Maha"]

# Initialize an empty list for ground data
ground_input = []

# Prepare the ground data for each location
for df in ground_dfs:
    ground_input.extend(prepare_ground_data(df, years, seasons))


# Check the shapes
for i, season_data in enumerate(satellite_input):
    print(f"Season {i+1}: Satellite input shape = {season_data.shape}, Ground input shape = {ground_input[i].shape}")

ground_input[-1]


Season 1: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 2: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 3: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 4: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 5: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 6: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 7: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 8: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 9: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 10: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 11: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 12: Satellite input shape = (6, 256, 256, 4), Ground input shape = (6, 4)
Season 13: Satellite input shape = (6

array([[28.1       , 20.80666667,  6.69333333,  8.27      ],
       [26.67741935, 20.08709677, 12.59032258,  7.62580645],
       [26.14666667, 19.86      , 15.75333333,  7.7       ],
       [25.34193548, 20.21612903, 13.34516129,  9.51290323],
       [25.0483871 , 18.77419355,  8.0516129 , 10.67096774],
       [26.34827586, 18.85172414,  3.66551724, 11.26896552]])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

X_satellite = np.array(satellite_input)  # Shape: (10, T, 128, 128, 4)
X_ground = np.array(ground_input)        # Shape: (10, T, 4)

T = None  # Let T be variable because the input sequence length varies


# model = create_stacked_lstm_model(cnn_input_shape, ground_input_shape, T)

# Split satellite data
# X_satellite_train, X_satellite_test, X_ground_train, X_ground_test, Y_train, Y_test = train_test_split(
#     X_satellite, X_ground, Y_target, test_size=0.2, random_state=42
# )

split_point = len(X_satellite) - 10

X_satellite_train = (X_satellite[:split_point])
X_ground_train = (X_ground[:split_point])
Y_train = (Y_target[:split_point])

X_satellite_test = (X_satellite[split_point:])
X_ground_test = (X_ground[split_point:])
Y_test = (Y_target[split_point:])
Y_test


array([1905.33075576, 2071.82767269, 1782.82797261, 2012.58983111,
       1966.44306809, 2059.12138496, 1224.88532415, 1355.41832843,
       1481.85811471, 1842.87500529])

In [ ]:
import numpy as np

# Print shapes for satellite input
print("Satellite Input (X_satellite):")
print(f"Shape of full dataset: {X_satellite.shape}")
print(f"Shape of training data: {X_satellite_train.shape}")
print(f"Shape of test data: {X_satellite_test.shape}")

# Uncomment and use if you include ground data
print("\nGround Input (X_ground):")
print(f"Shape of full dataset: {X_ground.shape}")
print(f"Shape of training data: {X_ground_train.shape}")
print(f"Shape of test data: {X_ground_test.shape}")

# Print shapes for target data
print("\nTarget Output (Y_target):")
print(f"Shape of full dataset: {Y_target.shape}")
print(f"Shape of training data: {Y_train.shape}")
print(f"Shape of test data: {Y_test.shape}")


Satellite Input (X_satellite):
Shape of full dataset: (50, 6, 256, 256, 4)
Shape of training data: (40, 6, 256, 256, 4)
Shape of test data: (10, 6, 256, 256, 4)

Ground Input (X_ground):
Shape of full dataset: (50, 6, 4)
Shape of training data: (40, 6, 4)
Shape of test data: (10, 6, 4)

Target Output (Y_target):
Shape of full dataset: (50,)
Shape of training data: (40,)
Shape of test data: (10,)


In [ ]:
#best model upto now

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    GlobalAveragePooling1D, LSTM, Concatenate
)

# Satellite Input
satellite_input = Input(shape=(6, 256, 256, 4), name="satellite_input")
time_distributed = TimeDistributed(
    Conv2D(32, (3, 3), activation='relu', padding='same'))(satellite_input)
time_distributed = TimeDistributed(MaxPooling2D((2, 2)))(time_distributed)
time_distributed = TimeDistributed(
    Conv2D(64, (3, 3), activation='relu', padding='same'))(time_distributed)
time_distributed = TimeDistributed(MaxPooling2D((2, 2)))(time_distributed)
time_distributed = TimeDistributed(
    Conv2D(128, (3, 3), activation='relu', padding='same'))(time_distributed)
time_distributed = TimeDistributed(Flatten())(time_distributed)
time_distributed = TimeDistributed(
    Dense(128, activation='relu'))(time_distributed)
satellite_features = GlobalAveragePooling1D()(time_distributed)

# Ground Input
ground_input = Input(shape=(6, 4), name="ground_input")
ground_features = LSTM(64, activation='relu', return_sequences=False)(ground_input)

# Merge Features
merged = Concatenate()([satellite_features, ground_features])

# Fully Connected Layers
fc = Dense(64, activation='relu')(merged)
fc = Dropout(0.3)(fc)
output = Dense(1, activation='linear', name="yield_output")(fc)

# Model
model = Model(inputs=[satellite_input, ground_input], outputs=output)

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ satellite_input           │ (None, 6, 256, 256, 4) │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 6, 256, 256,    │          1,184 │ satellite_input[0][0]  │
│ (TimeDistributed)         │ 32)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_1        │ (None, 6, 128, 128,    │              0 │ time_distributed[0][0] │
│ (TimeDistributed)         │ 32)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_2        │ (None, 6, 128, 128,    │         18,496 │ time_distributed_1[0]… │
│ (TimeDistributed)         │ 64)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_3        │ (None, 6, 64, 64, 64)  │              0 │ time_distributed_2[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_4        │ (None, 6, 64, 64, 128) │         73,856 │ time_distributed_3[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_5        │ (None, 6, 524288)      │              0 │ time_distributed_4[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_6        │ (None, 6, 128)         │     67,108,992 │ time_distributed_5[0]… │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ground_input (InputLayer) │ (None, 6, 4)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ time_distributed_6[0]… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64)             │         17,664 │ ground_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 192)            │              0 │ global_average_poolin… │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │         12,352 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ yield_output (Dense) 

 Total params: 67,232,609 (256.47 MB)

 Trainable params: 67,232,609 (256.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model with both inputs
history = model.fit(
    [X_satellite_train, X_ground_train], Y_train,
    validation_data=([X_satellite_test, X_ground_test], Y_test),
    epochs=50,
    batch_size=8
)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 109s 20s/step - loss: 3804431.0000 - mae: 1916.4838 - val_loss: 2246180.5000 - val_mae: 1471.1729
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 145s 21s/step - loss: 2356399.2500 - mae: 1421.1647 - val_loss: 146499.0781 - val_mae: 322.5835
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 136s 20s/step - loss: 1278326.2500 - mae: 978.9701 - val_loss: 482517.0000 - val_mae: 626.6315
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 101s 20s/step - loss: 798449.2500 - mae: 769.9598 - val_loss: 216905.6250 - val_mae: 342.0900
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 144s 21s/step - loss: 496209.1250 - mae: 565.7656 - val_loss: 108447.9531 - val_mae: 257.7870
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 144s 22s/step - loss: 559336.3750 - mae: 662.4788 - val_loss: 155776.7031 - val_mae: 359.6944
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 137s 20s/step - loss: 538311.9375 - mae: 637.7686 - val_loss: 339789.6562 - val_mae: 491.4088
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 100s 20s/step - loss: 441802.7812 - mae: 570.39

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/models/model3_best.keras')

print("Model saved successfully!")


Model saved successfully!


In [ ]:
# Evaluate the model on test data
loss, mae = model.evaluate([X_satellite_test, X_ground_test], Y_test, batch_size=8)

print(f"Test Loss: {loss:.4f}")
print(f"Test Mean Absolute Error (MAE): {mae:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 122833.8828 - mae: 228.1297
Test Loss: 122649.4141
Test Mean Absolute Error (MAE): 234.2745


In [ ]:
# Make predictions on test data
predictions = model.predict([X_satellite_test, X_ground_test])

# Print predictions and actual values
for i in range(10):  # Print the first 10 predictions
    print(f"Predicted Yield: {predictions[i][0]:.4f}, Actual Yield: {Y_test[i]:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Yield: 1946.6134, Actual Yield: 1905.3308
Predicted Yield: 2049.0671, Actual Yield: 2071.8277
Predicted Yield: 1952.2350, Actual Yield: 1782.8280
Predicted Yield: 1984.3073, Actual Yield: 2012.5898
Predicted Yield: 1971.2404, Actual Yield: 1966.4431
Predicted Yield: 1970.3925, Actual Yield: 2059.1214
Predicted Yield: 1963.2927, Actual Yield: 1224.8853
Predicted Yield: 1988.4747, Actual Yield: 1355.4183
Predicted Yield: 1949.7596, Actual Yield: 1481.8581
Predicted Yield: 1990.9958, Actual Yield: 1842.8750


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Define the new input shape
new_input_shape = (6, 256, 256, 4)  # Sequence length: 6, spatial dimensions: 256x256, channels: 4

# Load the existing model
# Assuming 'model' is the loaded model

# Adjust the input layer
new_input = tf.keras.layers.Input(shape=new_input_shape, name='new_input_layer')

# Resize the input to the expected shape for the TimeDistributed layer
reshaped_input = tf.keras.layers.TimeDistributed(
    tf.keras.layers.Resizing(33, 33),
    name='resize_layer'
)(new_input)

# Pass the resized input through the existing layers
x = reshaped_input
for i, layer in enumerate(model.layers[1:]):
    # Rename layers dynamically if needed to avoid duplicate names
    layer._name = f"{layer.name}_copy_{i}"
    x = layer(x)

# Create the updated model
updated_model = tf.keras.Model(inputs=new_input, outputs=x)

# Compile the updated model
updated_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Check the new model summary
updated_model.summary()

# Prepare the training and testing datasets
# Assuming X_satellite_train, X_satellite_test, Y_train, Y_test are already defined
X_satellite_train = X_satellite_train.astype('float32')  # Ensure proper dtype
X_satellite_test = X_satellite_test.astype('float32')

# Train the model
history = updated_model.fit(
    X_satellite_train,
    Y_train,
    validation_data=(X_satellite_test, Y_test),
    epochs=50,  # Adjust based on your needs
    batch_size=8,  # Adjust batch size for memory
    verbose=1
)

# Save the fine-tuned model
updated_model.save("fine_tuned_model.h5")


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ new_input_layer (InputLayer)         │ (None, 6, 256, 256, 4)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resize_layer (TimeDistributed)       │ (None, 6, 33, 33, 4)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 6, 128)              │       1,142,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ yield_output (Dense)                 │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,150,561 (4.39 MB)

 Trainable params: 1,150,561 (4.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 371ms/step - loss: 3964383.0000 - mae: 1958.9766 - val_loss: 3220352.7500 - val_mae: 1770.7500
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 300ms/step - loss: 4070173.5000 - mae: 1989.5765 - val_loss: 3220049.5000 - val_mae: 1770.6643
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 290ms/step - loss: 3864567.5000 - mae: 1926.1298 - val_loss: 3219704.5000 - val_mae: 1770.5668
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 297ms/step - loss: 3941096.5000 - mae: 1952.3585 - val_loss: 3218762.7500 - val_mae: 1770.3008
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - loss: 3782707.7500 - mae: 1905.2830 - val_loss: 3213768.7500 - val_mae: 1768.8896
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step - loss: 3856362.2500 - mae: 1934.8348 - val_loss: 3199840.2500 - val_mae: 1764.9475
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - loss: 3995714.2500 - mae: 1970.5658 - val_loss: 3165041.2500 - val_mae: 1755.0592
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - loss: 3662

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Masking
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping

# Define the CNN model for processing satellite images
def create_cnn_model(input_shape):
    cnn_model = models.Sequential()
    cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    cnn_model.add(layers.MaxPooling2D((2, 2)))
    cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    cnn_model.add(layers.MaxPooling2D((2, 2)))
    cnn_model.add(layers.Flatten())
    return cnn_model

# Define the Stacked LSTM model (without ground data)
# Define the Stacked LSTM model without ground data
def create_stacked_lstm_model(cnn_input_shape, T):
    cnn_model = create_cnn_model(cnn_input_shape)  # CNN for satellite images

    # Define input layer for satellite images
    satellite_input = layers.Input(shape=(T,) + cnn_input_shape, name='satellite_input')

    # Process each time step individually
    cnn_features = layers.TimeDistributed(cnn_model)(satellite_input)

    # Masking and LSTM layers
    masked_features = Masking()(cnn_features)
    lstm_1 = layers.LSTM(256, return_sequences=True, use_cudnn=False)(masked_features)
    lstm_2 = layers.LSTM(128, use_cudnn=False)(lstm_1)

    # Output layer
    output = layers.Dense(1, activation='linear', name='output')(lstm_2)

    # Define the model
    model = models.Model(inputs=satellite_input, outputs=output)
    return model



# STEP 1: Load the pre-trained model
pretrained_model = load_model('/content/drive/MyDrive/models/model2.keras')  # Replace 'xx.keras' with your model path
print("Pre-trained model loaded successfully.")

# STEP 2: Freeze specific layers
for layer in pretrained_model.layers:
    if 'fusion_layer' not in layer.name:  # Freeze all except the fusion layer and beyond
        layer.trainable = False

print("Pre-trained layers frozen except for the last fusion and LSTM layers.")

# STEP 3: Modify and fine-tune the model
cnn_input_shape = (256, 256, 4)  # Satellite image input shape
T = 6                            # Number of time steps

# Create a new fine-tuned model based on architecture
fine_tuned_model = create_stacked_lstm_model(cnn_input_shape, T)

# Transfer weights from pre-trained model
# Transfer weights from pre-trained model to the new model
for old_layer, new_layer in zip(pretrained_model.layers, fine_tuned_model.layers):
    try:
        # Transfer weights only if layer names and weights match
        if old_layer.name == new_layer.name and len(old_layer.get_weights()) == len(new_layer.get_weights()):
            new_layer.set_weights(old_layer.get_weights())
            print(f"Weights transferred for layer: {new_layer.name}")
        else:
            print(f"Skipping weight transfer for layer: {old_layer.name}")
    except ValueError as e:
        print(f"Error transferring weights for layer {old_layer.name}: {e}")


print("Weights transferred to the new model.")

# STEP 4: Compile the fine-tuned model
fine_tuned_model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Use a low learning rate for fine-tuning
    loss=MeanSquaredError(),
    metrics=['mae']  # Mean Absolute Error
)

# STEP 5: Prepare data (replace with your actual data pipelines)
import numpy as np

# Example dummy data
train_satellite = X_satellite_train  # 100 samples of satellite input
train_yield = Y_train                  # 100 target yield values

val_satellite =  X_satellite_test    # Validation data
val_yield = Y_test                    # Validation target yield values

# STEP 6: Train the fine-tuned model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = fine_tuned_model.fit(
    X_satellite_train,
    Y_train,
    validation_data=(X_satellite_test, Y_test),
    epochs=20,
    batch_size=8,
    callbacks=[early_stopping]
)

# STEP 7: Evaluate the model
results = fine_tuned_model.evaluate(val_satellite, val_yield)
print(f"Validation Loss: {results[0]}, Validation MAE: {results[1]}")

# STEP 8: Save the fine-tuned model
fine_tuned_model.save('fine_tuned_model.keras')
print("Fine-tuned model saved successfully.")


Pre-trained model loaded successfully.
Pre-trained layers frozen except for the last fusion and LSTM layers.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Skipping weight transfer for layer: input_layer_1
Skipping weight transfer for layer: time_distributed
Skipping weight transfer for layer: global_average_pooling1d
Skipping weight transfer for layer: dense_1
Skipping weight transfer for layer: dropout
Skipping weight transfer for layer: yield_output
Weights transferred to the new model.
Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 101s 19s/step - loss: 3927345.7500 - mae: 1943.3955 - val_loss: 3202248.7500 - val_mae: 1765.6270
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 93s 19s/step - loss: 3951187.7500 - mae: 1949.0530 - val_loss: 3196691.0000 - val_mae: 1764.0511
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 141s 19s/step - loss: 3941819.5000 - mae: 1952.1870 - val_loss: 3192829.5000 - val_mae: 1762.9570
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 141s 19s/step - loss: 3930986.0000 - mae: 1948.3789 - val_loss: 3189532.7500 - val_mae: 1762.0228
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 140s 18s/step - loss: 3879786.0000 - mae: 1927.8344 - val_loss: 3186619.7500 - val_mae: 17

In [ ]:
tf.keras.backend.clear_session()



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import smart_resize



# Resize function for your input data
def resize_inputs(input_tensor, target_size=(33, 33)):
    return tf.map_fn(lambda x: tf.image.resize(x, target_size), input_tensor)

# === Step 1: Load Pre-trained CNN Model ===
# Load pre-trained model and remove the output layer
pretrained_model = load_model('/content/drive/MyDrive/models/model2.keras')

# Print summary to inspect layers
pretrained_model.summary()

# Extract CNN feature extractor layers without input/output layers
cnn_feature_extractor = pretrained_model.layers[-2].output  # Remove the last dense layer

# === Step 2: Define New Input Shape ===
input_shape = (6, 256, 256, 4)  # New dataset input shape
inputs = Input(shape=input_shape, name="input_layer")

# Wrap CNN feature extractor with TimeDistributed
x = TimeDistributed(tf.keras.layers.Lambda(lambda img: smart_resize(img, (33, 33))))(inputs)

# Flatten TimeDistributed output
x = TimeDistributed(Flatten())(x)

# Add LSTM Layers for temporal learning
x = LSTM(128, return_sequences=True, name="lstm_1")(x)
x = LSTM(64, return_sequences=False, name="lstm_2")(x)

# Fully Connected Layer
x = Dense(128, activation='relu', name="dense_1")(x)
x = Dropout(0.3, name="dropout_1")(x)

# Output Layer (Adjust activation for regression/classification)
outputs = Dense(1, activation='linear', name="output_layer")(x)

# Combine into Model
model = Model(inputs, outputs, name="cnn_lstm_transfer_learning")

# === Step 3: Compile the Model ===
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

# === Step 4: Train the Model ===
import numpy as np
x_train = X_satellite_train  # Replace with your training data
y_train = Y_train               # Replace with target values

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=8,
    validation_split=0.2
)

# Save Model
model.save('fine_tuned_model.h5')
print("Model fine-tuned successfully.")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 72, 33, 33, 4)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 72, 128)             │       1,142,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ yield_output (Dense)                 │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,451,685 (13.17 MB)

 Trainable params: 1,150,561 (4.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,301,124 (8.78 MB)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - loss: 4146039.2500 - mae: 2008.4478 - val_loss: 3041653.5000 - val_mae: 1707.5093
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 4186754.7500 - mae: 2015.2522 - val_loss: 3041297.5000 - val_mae: 1707.4023
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 4279805.0000 - mae: 2039.2330 - val_loss: 3040915.0000 - val_mae: 1707.2881
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 4110905.7500 - mae: 1998.1061 - val_loss: 3040453.0000 - val_mae: 1707.1501
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 4241173.0000 - mae: 2026.4926 - val_loss: 3039892.0000 - val_mae: 1706.9829
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 4304406.0000 - mae: 2048.5886 - val_loss: 3039267.0000 - val_mae: 1706.7976
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 4184796.5000 - mae: 2007.4993 - val_loss: 3038548.7500 - val_mae: 1706.5853
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 3957573.250

Model fine-tuned successfully.


In [ ]:
# X_satellite_train
Y_train

array([2679.82203617, 2317.28017421, 2550.45923003, 2422.29913044,
       2380.82762505, 2352.72251743, 1606.31745511, 1866.58856121,
       2212.76128725, 2058.60540505, 2053.71162345, 2230.22648242,
       2186.97648   , 2122.35815189, 2112.55637329, 2201.74433331,
       1255.6830813 , 1735.42527711, 1787.22812632, 1990.36186779,
       2068.85708158, 2232.03103112, 1953.46562941, 2174.56192191,
       2115.81724283, 1995.49921775, 1077.4536236 , 1385.92372624,
       1415.31802951, 1953.29873933, 1716.20971907, 2014.52469356,
       1770.34622289, 2132.01870763, 1719.73780966, 2131.40610003,
       1077.31227568, 1629.36273847, 1270.06261088, 1930.70878155])